# Information Retrieval Final Project

#### Ali Bazshoushtari

--------------------------------------------------------------------------------------------------------------------------------------------------------

## Imports

In [1]:
import parsivar
from string import punctuation
import json
import re
import numpy as np
from pathlib import Path
from collections import Counter
import collections

## Load Documents

In [2]:
def read_docs(): 
    docs = {}
    contents = []
    urls = []
    with open("IR_data_news_12k.json", 'r') as file:
        articles = json.load(file)
        for doc_id in articles.keys():
            # index of files
            index = str(doc_id)
            # extract and save url, title and content of each doc
            docs[index] = {'title': articles[index]['title'],
                             'content': articles[index]['content'],
                             'url': articles[index]['url'],
                            }
            articles[index].pop('tags')
            articles[index].pop('date')
            articles[index].pop('category')
            
            contents.append(docs[index]['content'])
            urls.append(docs[index]['url'])
    return docs, contents, urls, articles

In [3]:
docs, contents, urls, articles = read_docs()
print(docs['0'])
articles['0']

{'title': 'اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا', 'content': '\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان\xa0 قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/\n\n\n', 'url': 'https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا'}


{'title': 'اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا',
 'content': '\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان\xa0 قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/\n\n\n',
 'url': 'https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا'}

In [4]:
import re

class DataNormalization:
    def __init__(self):
        # pattern for matching mi in start of token
        self.mi_patterns = r"\bن?می[آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی]+"
        
        # punctuation marks
        self.punc_after = r"\.:!،؛؟»\]\)\}"
        self.punc_before = r"«\[\(\{"
        self.all_punc_marks = r"[\.:!،؛؟»\'\]\)\}|«\[\(\/\{><+\-?!=_]"

        self.number_not_persian = "0123456789%٠١٢٣٤٥٦٧٨٩"
        self.number_persian = "۰۱۲۳۴۵۶۷۸۹٪۰۱۲۳۴۵۶۷۸۹"
        
        # fathe kasre ,....
        self.arabic_patterns = [
                    ("[\u064b\u064c\u064d\u064e\u064f\u0650\u0651\u0652]", ""),
                ]
        
        self.punctuation_spacing_patterns = [
                # remove space before and after quotation
                ('" ([^\n"]+) "', r'"\1"'),
                (" ([" + self.punc_after + "])", r"\1"),  # remove space before
                ("([" + self.punc_before + "]) ", r"\1"),  # remove space after
                # put space after . and :
                (
                    "([" + self.punc_after[:3] + "])([^ " + self.punc_after + r"\d۰۱۲۳۴۵۶۷۸۹])",
                    r"\1 \2",
                ),
                (
                    "([" + self.punc_after[3:] + "])([^ " + self.punc_after + "])",
                    r"\1 \2",
                ),  # put space after
                (
                    "([^ " + self.punc_before + "])([" + self.punc_before + "])",
                    r"\1 \2",
                ),  # put space before
                # put space after number
                (r"(\d)([آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی])", r"\1 \2"),
                # put space before number
                (r"([آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی])(\d)", r"\1 \2"),
            ]
        
        # extra space patterns
        self.extra_space_patterns = [
            (r" {2,}", " "),           # remove extra spaces
            (r"\n{3,}", "\n\n"),       # remove extra newlines
            (r"\u200c{2,}", "\u200c"), # remove extra ZWNJs
            (r"\u200c{1,} ", " "),     # remove unneeded ZWNJs before space
            (r" \u200c{1,}", " "),     # remove unneeded ZWNJs after space
            (r"\b\u200c*\B", " "),      # remove unneeded ZWNJs at the beginning of words
            (r"\B\u200c*\b", " "),      # remove unneeded ZWNJs at the end of words
            (r"[ـ\r]", " "),           # remove keshide, carriage returns
        ]

        self.spacing_patterns = [
            (r"\xa0"," "),  # remove no-break char
            (r"([^ ]) ی ", r"\1‌ی "),          # fix 'ی' space
            (r"(^| )(ن?می) ", r"\1\2‌"),        # fix 'می' and 'نمی' space. put ZWNJ after them
            # put zwnj before تر, تری, ترین, گر, گری, ها, های
            (r"(?<=[^\n\d" + self.punc_after + self.punc_before + r"]{2}) (تر(ین?)?|گری?|های?)(?=[ \n" + self.punc_after + self.punc_before + r"]|$)", r"‌\1"),
            # fix suffix spacing ام, ایم, اش, اند, ای, اید, ات
            (r"([^ ]ه) (ا(م|یم|ش|ند|ی|ید|ت))(?=[ \n" + self.punc_after + r"]|$)", r"\1‌\2"),  # fix verb conjugation spacing
            (r"(ه)(ها)", r"\1‌\2"),  
        ]

        # some special unicodes which should be replaced by persian terms
        self.unicode_replacements = [
                    ("﷽"," بسم الله الرحمن الرحیم "),
                    ('ﷻ', "الله-جل-جلاله"),
                    ('ﷺ', "صلی-الله-علیه-وسلم"),
                    (" ﷼", " ریال"),
                    ("(ﷰ|ﷹ)", " صلی "),
                    (" ﷲ", " الله"),
                    (" ﷳ", " اکبر"),
                    (" ﷴ", " محمد"),
                    (" ﷵ", " صلعم"),
                    (" ﷶ", " رسول"),
                    (" ﷷ", " علیه"),
                    (" ﷸ", " وسلم"),
                    (" ﻵ|ﻶ|ﻷ|ﻸ|ﻹ|ﻺ|ﻻ|ﻼ", " لا"),
                    ("آ", "ا"),
                    ('أ', 'ا'),
                    ('ٱ', 'ا'),
                    ('إ', 'ا'),
                    ('ك','ک'),
                    ('ي','ی'),
                    ('ئ', 'ی'),
                    ('یٰ', 'ی'),
                    ('هٔ','ه'),
                    ('ؤ', 'و'),
                    ('%', "درصد"),
                    ('٪', "درصد")
                ]
        
        
        # bons of verbs
        with Path('verbs.dat').open(encoding="utf8") as verbs_file:
                verbs = list(
                    reversed([verb.strip() for verb in verbs_file if verb]),
                )
                self.present_bons = {verb[1:].split("#")[0].strip() for verb in verbs[1:]}
                self.past_bons = {verb.split("#")[1] for verb in verbs}


    @staticmethod
    def regex_replace(patterns: list, text: str) -> str:
        for pattern, repl in patterns:
            text = re.sub(pattern, repl, text)
        return text

    # fix spacings
    def spacing_correction(self, text: str) -> str:
        text = self.regex_replace(self.extra_space_patterns, text)
        text = self.regex_replace(self.punctuation_spacing_patterns, text)
        text = self.regex_replace(self.spacing_patterns, text)
        return text

    # replace special characters
    def unicode_replacement(cls, text: str) -> str:
        for old, new in cls.unicode_replacements:
            text = re.sub(old, new, text)
        return text


    # remove punctuation marks
    def remove_punc_marks(cls, text: str) -> str:
        return re.sub(cls.all_punc_marks, "", text)        
     
    # remove some arabic chars
    def remove_arabic_chars(cls, text: str) -> str:
        return cls.regex_replace(cls.arabic_patterns, text)

    
    # remove punctuation marks and arabic chars
    def remove_special_chars(cls, text: str) -> str:
        text = cls.remove_punc_marks(text)
        text = cls.remove_arabic_chars(text)
        return text
    
    # convert numbers to persian numbers
    def persian_number(cls, text: str) -> str:
        translation_table = str.maketrans(
            cls.number_not_persian,
            cls.number_persian )
        translated_text = text.translate(translation_table)
        return translated_text

    # seperate mi in start of verbs
    def seperate_mi(cls, text:str) -> str:
        matches = re.findall(cls.mi_patterns, text)
        for m in matches:
            r = re.sub("^(ن?می)", r"\1‌", m)
            # remove mi from token to check it contains the bon of a verb or not
            x = re.sub("^(ن?می)", "", m)
            for verb in cls.present_bons:
                if verb in x:
                    text = text.replace(m, r)
            for verb in cls.past_bons:
                if verb in x:
                    text = text.replace(m, r)
        return text

    # general normalization method to perform all above functions
    def normalize(cls, text:str) -> str:
        text = cls.remove_special_chars(text)
        text = cls.seperate_mi(text)
        text = cls.persian_number(text)
        text = cls.unicode_replacement(text)
        text = cls.spacing_correction(text)
        return text

## Normalization Test

In [5]:
normalizer = DataNormalization()
print(normalizer.seperate_mi("میرفتم میرفتم"))
print(normalizer.spacing_correction("به نام های خدا‌ ی  درباره ی مهربان ترین"))
print(normalizer.normalize("به نام های  ﷲ خدا‌ ی 99 % درباره ی ً ٌ ٍسلامَ ُ ِ ّ ْ+ئئئ  مقابله أ ﷽مهربان >. ﷽  . ترین"))
print(normalizer.normalize("«آزمون! رفت ? العين التي تمتل بك لن تنظر لغيرك...»"))
print(normalizer.normalize("'ابقَ قویّا، خالهٔ فَقِصّتُکَ لم تَنتَهی بعد ..' "))


می‌رفتم می‌رفتم
به نام‌های خدا‌ی درباره‌ی مهربان‌ترین
به نام‌های الله خدا‌ی ۹۹ درصد درباره‌ی سلام ییی مقابله ا بسم الله الرحمن الرحیم مهربان بسم الله الرحمن الرحیم‌ترین
ازمون رفت العین التی تمتل بک لن تنظر لغیرک
ابق قویا خاله فقصتک لم تنتهی بعد 


## Preprocessing

In [6]:
class DataPreprocessing:
    top_k = {}
    pattern = re.compile(r'([؟!?]+|[\d.:]+|[:.،؛»\])}"«\[({/\\])')
    after_verbs = {
                "ام",
                "ای",
                "است",
                "ایم",
                "اید",
                "اند",
                "بودم",
                "بودی",
                "بود",
                "بودیم",
                "بودید",
                "بودند",
                "باشم",
                "باشی",
                "باشد",
                "باشیم",
                "باشید",
                "باشند",
                   "شده",
            "نشده",
                "شوم",
                "شوی",
                "شود",
                "شویم",
                "شوید",
                "شوند",
                "شدم",
                "شدی",
                "شد",
                "شدیم",
                "شدید",
                "شدند",
                "نشوم",
                "نشوی",
                "نشود",
                "نشویم",
                "نشوید",
                "نشوند",
                "نشدم",
                "نشدی",
                "نشد",
                "نشدیم",
                "نشدید",
                "نشدند",
                "می‌شوم",
                "می‌شوی",
                "می‌شود",
                "می‌شویم",
                "می‌شوید",
                "می‌شوند",
                "می‌شدم",
                "می‌شدی",
                "می‌شد",
                "می‌شدیم",
                "می‌شدید",
                "می‌شدند",
                "نمی‌شوم",
                "نمی‌شوی",
                "نمی‌شود",
                "نمی‌شویم",
                "نمی‌شوید",
                "نمی‌شوند",
                "نمی‌شدم",
                "نمی‌شدی",
                "نمی‌شد",
                "نمی‌شدیم",
                "نمی‌شدید",
                "نمی‌شدند",
               
            }

    before_verbs = {
                "خواهم",
                "خواهی",
                "خواهد",
                "خواهیم",
                "خواهید",
                "خواهند",
                "نخواهم",
                "نخواهی",
                "نخواهد",
                "نخواهیم",
                "نخواهید",
                "نخواهند",
            }
    
    def __init__(self):
        # save terms like گفته ، خورده which are bon mazi + ه
        with Path('verbs.dat').open(encoding="utf8") as verbs_file:
                verbs = list(
                    reversed([verb.strip() for verb in verbs_file if verb]),
                )
                DataPreprocessing.verbe = {(verb.split("#")[0] + 'ه') for verb in verbs}
     
     
    # Remove Punctuations
    @staticmethod
    def Remove_Punctuations(text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)
    
    # Tokenization
    @staticmethod
    def Tokenization(text):
        text = DataPreprocessing.pattern.sub(r" \1 ", text.replace("\n", " ").replace("\t", " "))
        tokens = [word for word in text.split(" ") if word]
        tokens_cleaned = [token.strip('\xa0') for token in tokens if len(token.strip()) != 0]

        result = [""]
        # merge multi term verbs like خواهم رفت to خواهم_رفت
        for token in reversed(tokens_cleaned):
            if token in DataPreprocessing.before_verbs or (
                result[-1] in DataPreprocessing.after_verbs and token in DataPreprocessing.verbe
            ):
                result[-1] = token + "_" + result[-1]
            else:
                result.append(token)
        return list(reversed(result[1:]))
    
    # method to tokenize a text
    def tokenize(self, text):
        return self.Tokenization(text)
    
    # Normalization
    @staticmethod
    def Normalization(text):
        my_normalizer = DataNormalization()
        return my_normalizer.normalize(text)
    
    # Stemming
    @staticmethod
    def Stemming(tokens):
        stemmed = []
        my_stemmer = parsivar.FindStems()
        for token in tokens:
            stemmed.append(my_stemmer.convert_to_stem(token))
        return stemmed
    
    # Stop_Words
    @staticmethod
    def Top_K_Frequent(tokens,k):
        token_counts = Counter(tokens)
        sorted_tokens = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
        stopwords_to_remove = [token for token, count in sorted_tokens[:k]]
        report = {token: count for token, count in sorted_tokens[:k]}
        return report
        
    # print top k frequent terms
    def print_top_k(self):
        for token, count in self.top_k.items():
            print(f"Token: {token}, Count: {count}")
            

    # preprocess a text and return final tokens
    def simple_preprocess(self, content):
        punctuated_content = self.Remove_Punctuations(content)
        normalized_content = self.Normalization(punctuated_content)
        tokens_of_a_sentence = self.Tokenization(normalized_content)
        final_tokens_of_a_sentence = self.Stemming(tokens_of_a_sentence)
        tokens = [token for token in final_tokens_of_a_sentence if token not in self.top_k]
        return tokens
        

    # preprocess all given docs
    def preprocess(self, docs, k=50):
        tokens = []
        counter = 0
        for idx in docs.keys():
            content = docs[str(idx)]['content']
            punctuated_content = self.Remove_Punctuations(content)            
            normalized_content = self.Normalization(punctuated_content)
            all_tokens = self.Tokenization(normalized_content)
            stemmed_tokens = self.Stemming(all_tokens)
            docs[str(idx)]['content'] = stemmed_tokens
            tokens += stemmed_tokens
            counter += 1
            # print progress
            if counter % 1000 == 0:
                print(counter, ' docs processed')
        # save top k frequent
        self.top_k = self.Top_K_Frequent(tokens, k)
        # remove stop words from doc tokens
        for doc_id, doc_content in docs.items():
            docs[doc_id]['content'] = [token for token in doc_content['content'] if token not in self.top_k]
        return docs

In [7]:
global preprocessor
preprocessor = DataPreprocessing()

### Load and Preprocess Docs

In [8]:
docs, contents, urls, articles = read_docs()

In [9]:
pre_processed_docs = preprocessor.preprocess(docs)

1000  docs processed
2000  docs processed
3000  docs processed
4000  docs processed
5000  docs processed
6000  docs processed
7000  docs processed
8000  docs processed
9000  docs processed
10000  docs processed
11000  docs processed
12000  docs processed


### Print Stop Words

In [10]:
preprocessor.print_top_k()

Token: و, Count: 219205
Token: در, Count: 164334
Token: به, Count: 133277
Token: از, Count: 92933
Token: این, Count: 82977
Token: که, Count: 76240
Token: با, Count: 68996
Token: را, Count: 67490
Token: اس, Count: 45541
Token: کرد&کن, Count: 45104
Token: برای, Count: 31022
Token: داشت&دار, Count: 30052
Token: تیم, Count: 27692
Token: شد&شو, Count: 26688
Token: ان, Count: 26332
Token: کرد, Count: 22938
Token: هم, Count: 22393
Token: کشور, Count: 21730
Token: ما, Count: 19717
Token: یک, Count: 18733
Token: بود&باش, Count: 18275
Token: بازی, Count: 17796
Token: باید, Count: 16196
Token: تا, Count: 15870
Token: بر, Count: 15685
Token: شد, Count: 15641
Token: وی, Count: 15486
Token: داد&ده, Count: 14849
Token: خود, Count: 14552
Token: مجلس, Count: 14520
Token: اسلامی, Count: 14415
Token: گزارش, Count: 14040
Token: فارس, Count: 13969
Token: گفت, Count: 13773
Token: مردم, Count: 13201
Token: پیام, Count: 13112
Token: رییس, Count: 12793
Token: ایران, Count: 12717
Token: خبرگزاری, Count: 12429
T

### Example of Preprocessed Doc Content

In [11]:
pre_processed_docs['0']

{'title': 'اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا',
 'content': ['کنفدراسیون',
  'فوتبال',
  'اسیا',
  'AFC',
  'نامه',
  'رسمی',
  'فدراسیون',
  'فوتبال',
  'باشگاه',
  'گیتی',
  'پسند',
  'زمان',
  'قرعه',
  'کشید&کش',
  'جام',
  'باشگاه',
  'فوتسال',
  'اسیا',
  'رسما',
  'اعلام',
  'اساس',
  '۲۵',
  'فروردین',
  'ماه',
  '۱۴۰۱',
  'مراسم',
  'قرعه',
  'کشید&کش',
  'جام',
  'باشگاه',
  'فوتسال',
  'اسیا',
  'مالزی',
  'برگزار',
  'باشگاه',
  'گیتی',
  'پسند',
  'بعنوان',
  'قهرمان',
  'فوتسال',
  '۱۴۰۰',
  'مسابقات',
  'راه',
  'پیدا',
  'کرده_است',
  'پیش',
  'گیتی',
  'پسند',
  'تجربه',
  '۳',
  'دوره',
  'حضور',
  'جام',
  'باشگاه',
  'فوتسال',
  'اسیا',
  'داشته',
  'هر',
  'سه',
  'دوره',
  'فینال',
  'مسابقات',
  'راه',
  'پیدا',
  'کرده',
  'عنوان',
  'قهرمانی',
  'دو',
  'مقام',
  'دومی',
  'بدست',
  'اورده'],
 'url': 'https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا'}

## Positional Inverted Matrix

In [12]:
def positional_inverted_index(docs):
    p_inverted_index = {}
    for index in docs:
        for position, term in enumerate(docs[index]['content']):
            # if its not a new term
            if term in p_inverted_index:
                # if the doc has already been in posting list of that term
                if index in p_inverted_index[term]['docs']:
                    p_inverted_index[term]['docs'][index]['positions'].append(position)
                    p_inverted_index[term]['docs'][index]['tf_td'] += 1 # tf_td is term frequency of t in d
                else:
                    p_inverted_index[term]['doc_freq'] += 1
                    p_inverted_index[term]['docs'][index] = { 
                                                    'positions': [position],
                                                    'tf_td': 1
                                                    }
                p_inverted_index[term]['collection_frequency'] += 1
                
            # add term to dictionary if its new
            else:
                p_inverted_index[term] = {
                 'doc_freq': 1,
                 'collection_frequency': 1,
                 'docs': {  # Postings list
                       index: {
                           'positions': [position],
                           'tf_td': 1
                           }
                    }
                }
    return p_inverted_index

In [13]:
p_inverted_index = positional_inverted_index(pre_processed_docs)

In [14]:
print(len(p_inverted_index))

50182


In [15]:
tdf = []
for term in p_inverted_index:
    tdf += [(term, p_inverted_index[term]['collection_frequency'])]
sorted_tdf = sorted(tdf, key=lambda x: x[1])
print(sorted_tdf[:20])
print(sorted_tdf[30000:30020])
# print(sorted_tdf[:-20])

[('گفتتیم', 1), ('۳۸۲', 1), ('النا', 1), ('کنندالبته', 1), ('نواقصات', 1), ('اندیشیده_شد', 1), ('شیردختران', 1), ('فوتبالاسامی', 1), ('ذهبی', 1), ('نیافرد', 1), ('شیخ\u200cبهایی', 1), ('مزروعی', 1), ('تنیس\u200cباز', 1), ('شن\u200cریزی', 1), ('مانفیلس', 1), ('تنیس\u200cبازان', 1), ('کرگیوس', 1), ('تسیتسیپاس', 1), ('بروکسبی', 1), ('۶۸۹', 1)]
[('انتوان', 3), ('کاراسکو', 3), ('کیفی\u200cسازی', 3), ('استهمه', 3), ('ببند', 3), ('توانایی\u200cاش', 3), ('حزباوی', 3), ('گیلکی', 3), ('پورعروجی', 3), ('بازگردید', 3), ('لایک', 3), ('۴۶۹', 3), ('تبریزمس', 3), ('تهرانقشقایی', 3), ('پیونگ\u200cچانگ', 3), ('دوطرف', 3), ('امستردام', 3), ('یایونده', 3), ('خالیست', 3), ('ایگالو', 3)]


In [16]:
p_inverted_index['فوتبال']['collection_frequency']

8188

In [17]:
p_inverted_index['ببند']

{'doc_freq': 2,
 'collection_frequency': 3,
 'docs': {'2645': {'positions': [401], 'tf_td': 1},
  '8272': {'positions': [778, 850], 'tf_td': 2}}}

## Documents' Vectors & Champions list

Creating Documents' Vectors and also adding tf_idf score in positional inverted index for each document d in postings list of term t

In [18]:
N = len(pre_processed_docs)
print(N)

12202


In [19]:
# This function calculates tf_idf and add it to p_inverted_index, creates documents' vectors, and creates champions lists
def calculate_doc_tf_idf(p_inverted_index, N, champ_len): # N is number of documents
    docs_vectors = {}
    
    for term in p_inverted_index:
        term_docs = dict(p_inverted_index[term]['docs'])  # term_docs is actually postings list of the term
        df_t = p_inverted_index[term]['doc_freq']
        # calculating tf_idf for each doc in postings list
        for doc in p_inverted_index[term]['docs']:
            tf = p_inverted_index[term]['docs'][doc]['tf_td']
            w_td = (np.log10( N / df_t )) * (1 + np.log10(tf))
            p_inverted_index[term]['docs'][doc]['tf_idf'] = w_td
            
            # add weight of this term to docs vector for future usages in query processing
            if doc not in docs_vectors:
                docs_vectors[doc] = {}
            docs_vectors[doc][term] = {'tf_idf':w_td,'tf':tf}

            
        # sort postings list and put it in champions list of each term
        sorted_term_docs = sorted(term_docs, key=lambda doc: term_docs[doc]['tf_td'], reverse=True)
        p_inverted_index[term]['champions_list'] = {}
        for doc_number in sorted_term_docs[:champ_len] if champ_len < df_t else sorted_term_docs:
            p_inverted_index[term]['champions_list'][doc_number] = {
                'tf_td': p_inverted_index[term]['docs'][doc_number]['tf_td'],
                'tf_idf' : p_inverted_index[term]['docs'][doc_number]['tf_idf']
            }
       
    return p_inverted_index, docs_vectors

In [20]:
champ_len = 20
p_inverted_index, docs_vectors = calculate_doc_tf_idf(p_inverted_index, N, champ_len)

Testing 

In [21]:
docs_vectors['8535']

{'ماه': {'tf_idf': 0.7600951597276173, 'tf': 1},
 'مراسم': {'tf_idf': 1.2864016614122344, 'tf': 1},
 'برگزار': {'tf_idf': 0.8213161665267152, 'tf': 2},
 'عنوان': {'tf_idf': 0.4918175114962707, 'tf': 1},
 'مقام': {'tf_idf': 1.11330316705667, 'tf': 1},
 'جوان': {'tf_idf': 0.938136923221623, 'tf': 1},
 'برتر': {'tf_idf': 1.0520033409128284, 'tf': 2},
 'سازمان': {'tf_idf': 0.8321246883250831, 'tf': 1},
 'سوم': {'tf_idf': 1.0765553869442086, 'tf': 1},
 'زیر': {'tf_idf': 0.805625092262702, 'tf': 1},
 'برنامه': {'tf_idf': 0.6990411943176393, 'tf': 1},
 'ورزش': {'tf_idf': 1.1264361823279525, 'tf': 1},
 'امور': {'tf_idf': 1.012712670310246, 'tf': 1},
 'پذیرفت&پذیر': {'tf_idf': 1.3192751545741883, 'tf': 1},
 'رقابت': {'tf_idf': 0.8514078611611452, 'tf': 1},
 'نه': {'tf_idf': 0.9802401233929534, 'tf': 1},
 'میزبانی': {'tf_idf': 1.4060955072418055, 'tf': 1},
 'نیرو': {'tf_idf': 1.2498013704060005, 'tf': 2},
 'خانواده': {'tf_idf': 1.2189635327973172, 'tf': 1},
 'علی': {'tf_idf': 0.9135363229041925,

In [22]:
p_inverted_index['ازمون']['champions_list']

{'2976': {'tf_td': 35, 'tf_idf': 3.946197821263585},
 '3209': {'tf_td': 27, 'tf_idf': 3.7713780534017833},
 '3371': {'tf_td': 24, 'tf_idf': 3.6920334882741845},
 '11741': {'tf_td': 23, 'tf_idf': 3.6633631956531896},
 '10532': {'tf_td': 20, 'tf_idf': 3.5692125356931492},
 '3005': {'tf_td': 18, 'tf_idf': 3.4982364036050075},
 '3550': {'tf_td': 18, 'tf_idf': 3.4982364036050075},
 '3145': {'tf_td': 14, 'tf_idf': 3.328938389581892},
 '5776': {'tf_td': 13, 'tf_idf': 3.2790155395699467},
 '3341': {'tf_td': 9, 'tf_idf': 3.0312976691390547},
 '3818': {'tf_td': 9, 'tf_idf': 3.0312976691390547},
 '8325': {'tf_td': 9, 'tf_idf': 3.0312976691390547},
 '3633': {'tf_td': 8, 'tf_idf': 2.9519531040114564},
 '4122': {'tf_td': 8, 'tf_idf': 2.9519531040114564},
 '4450': {'tf_td': 8, 'tf_idf': 2.9519531040114564},
 '4998': {'tf_td': 8, 'tf_idf': 2.9519531040114564},
 '5330': {'tf_td': 8, 'tf_idf': 2.9519531040114564},
 '11032': {'tf_td': 8, 'tf_idf': 2.9519531040114564},
 '3093': {'tf_td': 7, 'tf_idf': 2.86

In [23]:
p_inverted_index['ببند']

{'doc_freq': 2,
 'collection_frequency': 3,
 'docs': {'2645': {'positions': [401],
   'tf_td': 1,
   'tf_idf': 3.7854010249923875},
  '8272': {'positions': [778, 850], 'tf_td': 2, 'tf_idf': 4.924920279132276}},
 'champions_list': {'8272': {'tf_td': 2, 'tf_idf': 4.924920279132276},
  '2645': {'tf_td': 1, 'tf_idf': 3.7854010249923875}}}

# Query Processing

In [24]:
def calculate_query_tf_idf(tf_tq, N, df_t):
    tf = 1 + np.log10(tf_tq)
    idf = np.log10(N / df_t)
    return tf * idf

In [25]:
import math
def vector_length(vector_dict):
    length = math.sqrt(sum(tf_idf_value['tf_idf'] ** 2 for tf_idf_value in vector_dict.values()))
    return length

### Calculating cosine similarities

In [26]:
def query_scoring(query, total_number_of_docs, p_inverted_index, k, champions_list = False):
    cosine_scores = {}
    query_tokens = preprocessor.simple_preprocess(query)
    query_tokens_count = dict(Counter(query_tokens))    
    print(query_tokens_count)

    for term in query_tokens_count:
        if term in p_inverted_index:
            if champions_list and k <= champ_len: 
                term_docs = p_inverted_index[term]['champions_list']
            else:
                term_docs = p_inverted_index[term]['docs']
            df_t = p_inverted_index[term]['doc_freq']
            w_tq = calculate_query_tf_idf(query_tokens_count[term], total_number_of_docs, df_t)
            for doc in term_docs:
                w_td = term_docs[doc]['tf_idf']
                # update doc scores for cosines similarity
                if int(doc) in cosine_scores:
                    # update doc scores for cosines similarity
                    cosine_scores[int(doc)] += w_td * w_tq
                else:
                    cosine_scores[int(doc)] = w_td * w_tq
    # calculate cosine score by dividing by doc vector length
    for doc_number in cosine_scores:
        cosine_scores[doc_number] /= vector_length(docs_vectors[str(doc_number)])
    # sort scores for top k 
    sorted_doc_cosine = sorted(cosine_scores.items(), key=lambda x:x[1], reverse=True)
    return sorted_doc_cosine[:k]

#### ----------------------------------------------------------------------------

In [27]:
def print_results(results):
    dict_result = {}
    for rank, result in enumerate(results):
        # print(f'my\n {result}')
        doc_id = result[0]
        if doc_id is None:
            continue
        print(100*'-' + '\n')
        print(f'Rank: {rank + 1}')
        print(f'Score: {result[1]}')
        print(f'ID: {doc_id}')
        print(f'{docs[f"{doc_id}"]["title"]}')
        print(f'{docs[f"{doc_id}"]["url"]}')
        dict_result[rank + 1] = {'docID': doc_id,
                                 'title': docs[str(doc_id)]["title"],
                                 'url'  : docs[str(doc_id)]["url"]}
    return dict_result

In [28]:
import time

def query_search(query, result_numbers = 5, champion_list = False):
    start = time.time()
    results = query_scoring(query, N, p_inverted_index, result_numbers, champion_list)
    end = time.time()
    print(f'Time: {end - start}')
    if len(results) == 0:
        print("نتیجه ای یافت نشد")
    else:
        print("Cosine Scores:")
        print(100*'=')
        return print_results(results)

In [29]:
r1 = query_search('وزیر', result_numbers= 3, champion_list= False)

{'وزیر': 1}
Time: 0.20999789237976074
Cosine Scores:
----------------------------------------------------------------------------------------------------

Rank: 1
Score: 0.14329647678742605
ID: 12191
آغاز نشست غیرعلنی مجلس با حضور امیرعبداللهیان
https://www.farsnews.ir/news/14000724000697/آغاز-نشست-غیرعلنی-مجلس-با-حضور-امیرعبداللهیان
----------------------------------------------------------------------------------------------------

Rank: 2
Score: 0.13972084497123127
ID: 10893
زارعی کوشا استاندار کردستان شد
https://www.farsnews.ir/news/14000826000505/زارعی-کوشا-استاندار-کردستان-شد
----------------------------------------------------------------------------------------------------

Rank: 3
Score: 0.1373662946239786
ID: 10640
جزئیات برگزاری جلسه رأی اعتماد به وزیر پیشنهادی آموزش و پرورش
https://www.farsnews.ir/news/14000904000549/جزئیات-برگزاری-جلسه-رأی-اعتماد-به-وزیر-پیشنهادی-آموزش-و-پرورش


In [30]:
r1c = query_search('وزیر', result_numbers= 3, champion_list= True)

{'وزیر': 1}
Time: 0.02099752426147461
Cosine Scores:
----------------------------------------------------------------------------------------------------

Rank: 1
Score: 0.05450334231579835
ID: 10268
دستور کار کمیسیون‌های مجلس/ دیدار نمایندگان با وزیر خارجه و بررسی تخلفات روحانی و زنگنه در انعقاد قرارداد کرسنت
https://www.farsnews.ir/news/14000910000920/دستور-کار-کمیسیون‌های-مجلس-دیدار-نمایندگان-با-وزیر-خارجه-و-بررسی
----------------------------------------------------------------------------------------------------

Rank: 2
Score: 0.05054510548588114
ID: 10890
سومین وزیر پیشنهادی آموزش و پرورش در ایستگاه بهارستان/ از سرعت عمل رئیس جمهور در معرفی تا گزینه‌ای از بدنه فرهنگیان
https://www.farsnews.ir/news/14000826000559/سومین-وزیر-پیشنهادی-آموزش-و-پرورش-در-ایستگاه-بهارستان-از-سرعت-عمل
----------------------------------------------------------------------------------------------------

Rank: 3
Score: 0.0477964222342544
ID: 7478
اصلاحات بدون رتوش| مروری بر بدزبانی‌ها و اهانت‌های دولت روحان

In [31]:
r2 = query_search('سازمان لیگ', result_numbers= 3, champion_list= False)

{'سازمان': 1, 'لیگ': 1}
Time: 0.38475990295410156
Cosine Scores:
----------------------------------------------------------------------------------------------------

Rank: 1
Score: 0.19017963053603698
ID: 2
محل برگزاری نشست‌های خبری سرخابی‌ها؛ مجیدی در سازمان لیگ، گل‌محمدی در تمرین پرسپولیس
https://www.farsnews.ir/news/14001224000971/محل-برگزاری-نشست‌های-خبری-سرخابی‌ها-مجیدی-در-سازمان-لیگ-گل‌محمدی-در
----------------------------------------------------------------------------------------------------

Rank: 2
Score: 0.17547625862038246
ID: 2607
دیدار حساس لیگ برتر فوتسال لغو شد
https://www.farsnews.ir/news/14001119000691/دیدار-حساس-لیگ-برتر-فوتسال-لغو-شد
----------------------------------------------------------------------------------------------------

Rank: 3
Score: 0.17091165680175882
ID: 3789
دیدار هوادار و استقلال لغو شد
https://www.farsnews.ir/news/14001104000380/دیدار-هوادار-و-استقلال-لغو-شد


In [32]:
r3 = query_search('کریسمس',result_numbers= 3, champion_list= False)

{'کریسمس': 1}
Time: 0.017000198364257812
Cosine Scores:
----------------------------------------------------------------------------------------------------

Rank: 1
Score: 0.8669566044973201
ID: 5933
ستاره اسپانیایی؛ هدیه کریسمس گواردیولا به ژاوی+عکس
https://www.farsnews.ir/news/14001007000739/ستاره-اسپانیایی-هدیه-کریسمس-گواردیولا-به-ژاوی-عکس
----------------------------------------------------------------------------------------------------

Rank: 2
Score: 0.7741970328238605
ID: 6117
کی‌روش «دیکتاتور» لقب گرفت/اختلاف مرد پرتغالی با مصری‌ها به خاطر کریسمس+عکس
https://www.farsnews.ir/news/14001005000165/کی‌روش-دیکتاتور-لقب-گرفت-اختلاف-مرد-پرتغالی-با-مصری‌ها-به-خاطر-کریسمس
----------------------------------------------------------------------------------------------------

Rank: 3
Score: 0.7332855143304998
ID: 6120
کشتار در ورزشگاه فوتبال در آستانه سال جدید
https://www.farsnews.ir/news/14001005000143/کشتار-در-ورزشگاه-فوتبال-در-آستانه-سال-جدید


In [33]:
tf_idf_dc = p_inverted_index['کریسمس']['docs']['5933']['tf_idf']
score_d = tf_idf_dc / vector_length(docs_vectors['5933'])
print(tf_idf_dc, score_d, len(docs['5933']['content']))

3.0864310206563688 0.2808929144034299 36


In [34]:
tf_idf_dc2 = p_inverted_index['کریسمس']['docs']['6117']['tf_idf']
score_d2 = tf_idf_dc2 / vector_length(docs_vectors['6117'])
print(tf_idf_dc2, score_d2, len(docs['6117']['content']))

4.559032861837624 0.25083892289911525 129


In [35]:
r3c = query_search('کریسمس',result_numbers= 3, champion_list= True)

{'کریسمس': 1}
Time: 0.01399993896484375
Cosine Scores:
----------------------------------------------------------------------------------------------------

Rank: 1
Score: 0.8669566044973201
ID: 5933
ستاره اسپانیایی؛ هدیه کریسمس گواردیولا به ژاوی+عکس
https://www.farsnews.ir/news/14001007000739/ستاره-اسپانیایی-هدیه-کریسمس-گواردیولا-به-ژاوی-عکس
----------------------------------------------------------------------------------------------------

Rank: 2
Score: 0.7741970328238605
ID: 6117
کی‌روش «دیکتاتور» لقب گرفت/اختلاف مرد پرتغالی با مصری‌ها به خاطر کریسمس+عکس
https://www.farsnews.ir/news/14001005000165/کی‌روش-دیکتاتور-لقب-گرفت-اختلاف-مرد-پرتغالی-با-مصری‌ها-به-خاطر-کریسمس
----------------------------------------------------------------------------------------------------

Rank: 3
Score: 0.7332855143304998
ID: 6120
کشتار در ورزشگاه فوتبال در آستانه سال جدید
https://www.farsnews.ir/news/14001005000143/کشتار-در-ورزشگاه-فوتبال-در-آستانه-سال-جدید


In [36]:
r4 = query_search('کمیسیون اجتهاد', result_numbers = 5, champion_list = True)

{'کمیسیون': 1, 'اجتهاد': 1}
Time: 0.01900005340576172
Cosine Scores:
----------------------------------------------------------------------------------------------------

Rank: 1
Score: 0.41093649508138524
ID: 12141
«ماموستا عبدالسلام کریمی» مشاور رئیس جمهور در امور اقوام و اقلیت‌های دینی و مذهبی شد
https://www.farsnews.ir/news/14000725000846/ماموستا-عبدالسلام-کریمی-مشاور-رئیس-جمهور-در-امور-اقوام-و-اقلیت‌های
----------------------------------------------------------------------------------------------------

Rank: 2
Score: 0.2784609427492448
ID: 9816
تحقیر زن در اندیشه غرب
https://www.farsnews.ir/news/14000923000557/تحقیر-زن-در-اندیشه-غرب
----------------------------------------------------------------------------------------------------

Rank: 3
Score: 0.2641584981329873
ID: 12198
نقدی بر یادداشت «مرزبندی گفتمانی با طالبان»/ وارونه‌نمایی گفتمانی اصلاح‌طلبان
https://www.farsnews.ir/news/14000724000611/نقدی-بر-یادداشت-مرزبندی-گفتمانی-با-طالبان-وارونه‌نمایی-گفتمانی
----------------------

In [37]:
r4 = query_search('جیانی اینفانتینو',result_numbers= 3, champion_list= False)

{'جیانی': 1, 'اینفانتینو': 1}
Time: 0.040000200271606445
Cosine Scores:
----------------------------------------------------------------------------------------------------

Rank: 1
Score: 1.5289023053255142
ID: 6731
حضور مهدوی کیا در بازی نمادین ستارگان در ورزشگاه جام جهانی
https://www.farsnews.ir/news/14000926000283/حضور-مهدوی-کیا-در-بازی-نمادین-ستارگان-در-ورزشگاه-جام-جهانی
----------------------------------------------------------------------------------------------------

Rank: 2
Score: 1.2640737943569136
ID: 4033
احتمال لغو سفر اینفانتینو به تهران/ وضعیت دستیاران اسپانیایی شمسایی بررسی می شود
https://www.farsnews.ir/news/14001102000084/احتمال-لغو-سفر-اینفانتینو-به-تهران-وضعیت-دستیاران-اسپانیایی-شمسایی
----------------------------------------------------------------------------------------------------

Rank: 3
Score: 0.9527225123123577
ID: 3704
تکذیب دیدار رئیس فیفا با رییسی
https://www.farsnews.ir/news/14001105000360/تکذیب-دیدار-رئیس-فیفا-با-رییسی
